# FIRE 

This notebook will try to answer the following questions:

- Given my current savings and no income, how many months will my savings last


## Assumptions

- The annual return on the stock investment (assumption ETF) is an average and is applied monthly. 
The assumption is a long term.
- The bond investment has a fixed return rate that's applied monthly. In reality that should be close to inflation rate.
- The monthly income is constant and is added to cash every month. It's also a simplification as in reality the income is not constant
- The monthly expenses are constant but the inflation rate is applied. In reality, the expenses won't be constant as they change depending on many factors.
- The inflation rate is constant
- When the cash is depleted, the investment is used to cover the expenses. The investment is used in the following order: cash, bond, stock, real estate. 
- Simulation ends when the wealth is depleted and no more assets are available to cover the expenses.


In [7]:
import sys
from pathlib import Path

project_root = Path.cwd().parent 
src_path = project_root / 'src'

sys.path.append(str(src_path))

In [8]:
import datetime
from fire.simulations import FireSimulation, run_simulation, InvestmentProperty
from fire.inflation import random_inflation_gen, inflation_from_file_gen
import pandas as pd
import plotly.express as px
from decimal import Decimal


In [9]:
from decimal import Decimal

inf_gen = inflation_from_file_gen(project_root / 'data' / 'poland_monthly_cpi.csv', monthly=True)

init = FireSimulation(
    stock_investments=Decimal("100000"),
    bonds_investments=Decimal("100000"),
    cash=Decimal("100000"),
    investment_properties=[
        InvestmentProperty(
            mortgage_left=Decimal("292000"),
            mortgage_rate=Decimal("7.66"),
            mortgage_months=202,
            market_value=Decimal("520000"),
            monthly_income=Decimal("2100"),
        )
    ],
    stock_return_rate=Decimal("0.05"),
    bonds_return_rate=Decimal("0.04"),
    monthly_expenses=Decimal("16000"),
    annual_inflation_rate=Decimal("0.04"),
    monthly_income=Decimal("16000"),
    annual_property_appreciation_rate=Decimal("0.02"),
    invest_cash_surplus=True,
    invest_cash_threshold=Decimal("50000"),
    invest_cash_surplus_strategy='80-20',
    date=datetime.datetime.fromisoformat('2024-04-01')
)

# simulate for next X years
years = 20
expected_number_of_months = years * 12
simulation = run_simulation(init, years*12, inflation_rate_gen=inf_gen)

In [10]:

df = pd.DataFrame([s.to_dict() for s in simulation])

number_of_records = len(df)
last_record = df.iloc[-1]
df


,stock_investments,bonds_investments,cash,monthly_expenses,monthly_income,date,stock_return_rate,investment_properties,bonds_return_rate,annual_inflation_rate,...,invest_cash_surplus,invest_cash_threshold,invest_cash_surplus_strategy,liquid_wealth,wealth_inc_properties,properties_monthly_mortgage,properties_market_value,properties_monthly_income,properties_net_cash_value,properties_mortgage_left
0,100000.00,100000.00,100000.0,16000.00,16000.0,2024-04-01,0.05,"[{'market_value': 520000, 'monthly_income': 21...",0.04,0.040,...,True,50000.0,80-20,300000.00,528000.00,2576.48,520000.00,2100.0,228000.00,292000.00
1,142147.87,110766.13,50000.0,15936.00,16000.0,2024-05-01,0.05,"[{'market_value': 520866.67, 'monthly_income':...",0.04,-0.048,...,True,50000.0,80-20,302914.00,532493.22,2576.48,520866.67,2100.0,229579.22,291287.45
2,144471.35,111568.15,50000.0,15936.00,16000.0,2024-06-01,0.05,"[{'market_value': 521734.78, 'monthly_income':...",0.04,0.000,...,True,50000.0,80-20,306039.50,537203.93,2576.48,521734.78,2100.0,231164.43,290570.35
3,146830.01,112379.22,50000.0,15904.13,16000.0,2024-07-01,0.05,"[{'market_value': 522604.34, 'monthly_income':...",0.04,-0.024,...,True,50000.0,80-20,309209.23,541964.89,2576.48,522604.34,2100.0,232755.66,289848.68
4,149198.50,113192.99,50000.0,15904.13,16000.0,2024-08-01,0.05,"[{'market_value': 523475.35, 'monthly_income':...",0.04,0.000,...,True,50000.0,80-20,312391.49,546744.44,2576.48,523475.35,2100.0,234352.95,289122.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,47706.90,0.00,0.0,27029.14,16000.0,2038-10-01,0.05,[],0.04,0.300,...,True,50000.0,80-20,47706.90,47706.90,0.00,0.00,0.0,0.00,0.00
175,36849.51,0.00,0.0,27056.17,16000.0,2038-11-01,0.05,[],0.04,0.012,...,True,50000.0,80-20,36849.51,36849.51,0.00,0.00,0.0,0.00,0.00
176,25757.49,0.00,0.0,27245.56,16000.0,2038-12-01,0.05,[],0.04,0.084,...,True,50000.0,80-20,25757.49,25757.49,0.00,0.00,0.0,0.00,0.00
177,14128.83,0.00,0.0,27735.98,16000.0,2039-01-01,0.05,[],0.04,0.216,...,True,50000.0,80-20,14128.83,14128.83,0.00,0.00,0.0,0.00,0.00


In [11]:
print(f"Without any income and burning only your investments, you can survive for {number_of_records} months")



Without any income and burning only your investments, you can survive for 179 months


In [12]:
# I want to use plotly to create a nice stacked bar chart of bonds and stock investments
# with a line chart showing monthly expenses

fig = px.bar(df, x=df['date'], y=[
        "properties_net_cash_value",
        "stock_investments",
        "bonds_investments",
        "cash",
    ], title='Wealth over time')
fig.add_scatter(x=df['date'], y=df['monthly_expenses'], mode='lines', name='Monthly expenses')
fig.add_scatter(x=df['date'], y=df['wealth_inc_properties'], mode='lines', name='Wealth with net properties')

fig.show()


## Properties specific